## Oppstart

In [1]:
from dhlab.module_update import update, css

In [2]:
css()

In [3]:
import warnings
warnings.filterwarnings('ignore')

### Note that the file `nbtext.py` will be overwritten in the code below

This is probably ok, though. If not, change `update('nbtext', overwrite = True)` to `update('nbtext', overwrite = False)` and the file will stay unchanged. 


In [4]:
import dhlab.nbtext as nb
import dhlab.token_map as tm
from dhlab.nbtext import get_urn, make_graph, get_urn, relaterte_ord, navn, totals
import dhlab.graph_networkx_louvain as gnl
import pandas as pd
import networkx as nx
%matplotlib inline

plot = lambda x,  lw=3, alpha= 0.7: x.plot(figsize=(15,6), lw=lw, alpha=alpha)


# Hjelpekommandoer

`alle_korpusnavn` henter ut alle navn (eller forslag til) fra korpuset.
`tell_navn` teller opp navnene på tvers av korpuset 


In [5]:
import pandas as pd

def alle_korpusnavn(korpus):
    urner = nb.pure_urn(korpus)
    alle_navn = tm.combine_names(tm.corpus_names(urner))
    return alle_navn

def tell_navn(korpus, token_map, cutoff=1):
    res = dict()
    for urn in korpus['urn']:
        res[urn] = tm.count_name_strings(str(urn), token_map).to_dict()[0]
    return pd.DataFrame(pd.DataFrame(res).sum(axis=1).sort_values(ascending=False))

## Navn i bøker


Vi søker etter navn i en bok, og en serie av bøker.

Kommandoene er:
1. `names` henter et sett med forslag til navn, fra nbtext
1. `show_names` gir en oversikt over funnene, fra token_map
1. `names_to_token_map_file` lagrer navneforslagene til en excelfil (eller csv) for redigering (token_map)
1. `read_token_map_file` henter redigert fil tilbake for analyse (token_map)
1. `character_network` lager graf for navnene (token_map)
1. `show_graph`  tegner grafen (fra graph_networkx_louvain)
1. `show_communities` viser clustre i grafen (graph_networkx_louvain)


In [44]:
tm.combine_names??

In [6]:
#korpus = nb.book_corpus(author='Skram%')

In [7]:
korpus = nb.restore_metadata_from_excel('Realismekorpus.xls')

In [8]:
nb.pure_urn(korpus)

['2016091248055',
 '2008012902003',
 '2008072412004',
 '2012092706021',
 '2015102908007',
 '2008072312006',
 '2008072312008',
 '2016061508006',
 '2008072410002',
 '2012081009561',
 '2012112908109',
 '2015102208205',
 '2008072810001',
 '2014060406030',
 '2011011420015',
 '2014090908158',
 '2009102212006',
 '2007060400004',
 '2014111808073',
 '2016092848113',
 '2014100708075',
 '2009111612018',
 '2008040700041',
 '2012012008212',
 '2011062712001',
 '2014102708091',
 '2014090408112',
 '2006082500032',
 '2013031108025',
 '2015090108020',
 '2016111648042',
 '2016111048509',
 '2006081600054',
 '2014111308208',
 '2016111748516',
 '2012030208159',
 '2006082800025',
 '2016020208028',
 '2015062208117',
 '2015011608151',
 '2015120408050',
 '2015102208207',
 '2014030707087',
 '2016101248085',
 '2009020903024',
 '2016061748037',
 '2016040608138',
 '2016060848064',
 '2014090308001',
 '2016042848118',
 '2014012108004',
 '2016063008131',
 '2014032108017',
 '2014031108073',
 '2012101007041',
 '20151100

## Sjekk innholdet

Gjør noen søk for å se at alt er ok. Konkordanser er fine til det. Her går urnene gjennom `nb.pure_urn` -  i fremtiden skal det ikke være nødvendig.

In [24]:
nb.urn_concordance(word = 'Stavanger', urns = nb.pure_urn(korpus))

"""Journal for de smaa"", Kielland, Alexander L., 1999",tilbud som en sjøreise fra,Stavanger,til Lissabon og tok imot
"""Journal for de smaa"", Kielland, Alexander L., 1999",Victoria » ble bygd på,Stavanger,Støberi & Dok som byggenummer
"""Journal for de smaa"", Kielland, Alexander L., 1999","i firmaet Erik Berentsen ,",Stavanger,. Gunder Haave var skipper
"""Journal for de smaa"", Kielland, Alexander L., 1999","- Gandsfjorden , sjøveien fra",Stavanger,forbi Hillevåg til Sandnes .
"""Journal for de smaa"", Kielland, Alexander L., 1999",Christiansand og var hjemme i,Stavanger,7 de Juni . »
"Garman & worse, Kielland, Alexander L., 1969",Kiellands nå nedrevne hjem i,Stavanger,.
"Garman & worse, Kielland, Alexander L., 1969",Bredgaten i,Stavanger,.
"Garman & worse, Kielland, Alexander L., 1969",Gateparti fra,Stavanger,ca. 1895 ( « Issehuå
"Novelletter, Kielland, Alexander L., 1959",av det slags vi i,Stavanger,kaller sølvbryllupsregn .
"Novelletter, Kielland, Alexander L., 1959","En Skipperhistorie , skrevet i",Stavanger,"våren 1878 , like før"
"Skipper Worse, Kielland, Alexander L., 1973","bundet av Dreyer Aksjeselskap ,",Stavanger,"Papir fra Saugbrugsforeningen , Halden"



## Navn

Finn forslag til navn med `names`. Se på dem med `show_names`. Her er det muligheter til å skru litt på parametrene. Hvis ratioen skrus ned fjernes en del ord, og om cutoff settes høyere vil også enkelte navneforslag kunne forsvinne.

Velg hvilken bok som skal studeres (alle kan tas, men se under). Se på listen over korpus og sett den indeksen som passer. Her er det bare å føle seg frem.

In [10]:
korpus

,urn,forf,år,tittel,undertittel,forlag,sjanger,sesamid,språk,oversatt
1,2016091248055,"Bjørnson, Bjørnstjerne",1992,Bondefortellinger,NaN,Gyldendal,NaN,a729bd8342d6cf8ea4a73acedbdca60f,nob,NaN
2,2008012902003,"Bjørnson, Bjørnstjerne",1877,Brude-slaatten,NaN,Gyldendalske Boghandels Forlag,unknown,676654c5df2f9be0e0dc38b720052f7a,nob,NaN
3,2008072412004,"Bjørnson, Bjørnstjerne",1884,Det flager i byen og på havnen,NaN,Gyldendal,unknown,b60c34903e04edfd45cf4d5af720f923,dan,NaN
4,2012092706021,"Bjørnson, Bjørnstjerne",1968,Fiskerjenten,NaN,Gyldendal,fiction,4091da9c51f492bd3664f562177159b5,nob,NaN
5,2015102908007,"Bjørnson, Bjørnstjerne",1888,Godset,roman,Rössing,unknown,0e585c13f94908c440fc80bf09b6933b,dut,NaN
6,2008072312006,"Bjørnson, Bjørnstjerne",1879,Kaptejn Mansana,en Fortælling fra Italien,Gyldendal,unknown,a21c93bc745d5204724af3293c993662,nob,NaN
7,2008072312008,"Bjørnson, Bjørnstjerne",1877,Magnhild,en Fortælling,Gyldendal,unknown,bac744f8e3de71c02679c54468990254,nob,NaN
8,2016061508006,"Bjørnson, Bjørnstjerne",1922,Mary,fortælling,Gyldendal,unknown,95ebd955c6eb8c0a6c8146a61a6a2dd3,nob,NaN
9,2008072410002,"Bjørnson, Bjørnstjerne",1894,Nye Fortællinger,NaN,Gyldendal,unknown,de75a2072556c9fdcb7d5080496854cf,nob,NaN
10,2012081009561,"Bjørnson, Bjørnstjerne",1999,Paul Lange og Tora Parsberg,NaN,Nationaltheatret,drama,54d5cf43b7a65782b6a36739c8fa313e,nob,NaN


Finn korpuset ved å sette inn en index tilsvarende den som står helt til venstre i listen over.

Hent ut navn fra boken

In [36]:
total_names = tm.names(korpus.urn[27], cutoff=1)

In [39]:
tn27 = tm.count_name_strings(korpus.urn[27], tm.token_map(total_names))

In [43]:
tn27.loc['Stavanger_Papir']

0    5
Name: Stavanger_Papir, dtype: int64

In [41]:
tm.token_map(total_names)

[(('Aar',), ('Aar',)),
 (('Adam',), ('Adam', 'Kruse')),
 (('Ah',), ('Ah',)),
 (('Akk',), ('Akk',)),
 (('Amen',), ('Amen',)),
 (('Anden',), ('Anden',)),
 (('Arndt',), ('Johan', 'Arndt')),
 (('Bencke',), ('Bencke',)),
 (('Bergen',), ('Bergen',)),
 (('Bibelen',), ('Bibelen',)),
 (('Birgitte',), ('Birgitte',)),
 (('Boldeman',), ('Lauritz', 'Boldeman', 'Seehus')),
 (('Bolderå',), ('Bolderå',)),
 (('Bortgang',), ('Bortgang',)),
 (('Bremen',), ('Bremen',)),
 (('Bremerskib',), ('Bremerskib',)),
 (('Brev',), ('Brev',)),
 (('Broder',), ('Broder', 'Richard')),
 (('Brødre',), ('Brødre',)),
 (('By',), ('By',)),
 (('Byvågen',), ('Byvågen',)),
 (('Båt',), ('Fruens', 'Båt')),
 (('Bølgen',), ('Bølgen',)),
 (('Christian',), ('Christian', 'Fredrik')),
 (('Christum',), ('Christum',)),
 (('Dage',), ('Dage',)),
 (('Dem',), ('Dem',)),
 (('Deres',), ('Deres',)),
 (('Dermed',), ('Dermed',)),
 (('Død',), ('Moders', 'Død')),
 (('Eder',), ('Eder',)),
 (('Eders',), ('Eder',)),
 (('Egeland',), ('Erik', 'Pontoppidan

# Ta en titt på navneforslagene

Men redigere dem i excel for å se. Navnene er hentet fra hele korpuset.

In [35]:
total_names

(Counter({'Aar': 5,
          'Adam': 8,
          'Ah': 5,
          'Akk': 4,
          'Arndt': 3,
          'Bencke': 3,
          'Bergen': 8,
          'Bibelen': 8,
          'Birgitte': 7,
          'Bremen': 3,
          'Brev': 5,
          'Broder': 7,
          'Christian': 11,
          'Dage': 5,
          'Dem': 28,
          'Deres': 14,
          'Dermed': 3,
          'Død': 4,
          'Eder': 22,
          'Eders': 13,
          'Egeland': 21,
          'Endre': 13,
          'Er': 12,
          'Erik': 7,
          'Familiens': 17,
          'Fennefos': 66,
          'Firmaet': 3,
          'Flekkefjord': 3,
          'Flere': 4,
          'Forhold': 4,
          'Forretning': 3,
          'Francke': 7,
          'Franckes': 4,
          'Fredrik': 11,
          'Frelseren': 3,
          'Garmans': 6,
          'Gesvint': 8,
          'Gnadau': 4,
          'Grad': 3,
          'Grund': 6,
          'Gud': 41,
          'Guds': 29,
          'Hans': 109,
         

In [25]:
s = [x for x in total_names[0].keys() if x not in [z[0] for z in total_names[1].keys()] and x not in [z[1] for z in total_names[1].keys()]]

In [32]:
'Stavanger' in total_names[0].keys() 

False

### Lagring og redigering

Rediger filen fra cellen med lagringskommando (`names_to_token_map_file(....)`)
Listen trenger litt redigering, og sammenslåing. Lagre til Excel, og benytt `orient = column` for å ha navn i kolonnner, eller `orient = row` for å redigere radvis. Noen store filer med over 256 navn krever `row`. Kommandoen sjekker om filen allerede fins, og vil ikke skrive over. Følg med på output.

In [ ]:
#tm.names_to_token_map_file(total_names, filename = "Stedsnavn_Realismekorpus.xls", orient = "row")

### Redigeringsaktivitet

Åpne i for eksempel i excel eller tilsvarende som for eksempel openoffice eller libreoffice. Etter redigering, kjøres cellen under. Husk bare å benytte riktig filnavn. 

Husk at verdien på orient må være lik mellom lagring og lesing av excelfil.

In [13]:
redigerte_navn = tm.read_token_map_file("Stedsnavn_Realismekorpus.xls", orient = "row")

## Tell opp navn med hensyn til det som står i `redigerte_navn`

Bruker hjelpekommando `tell_navn`. Det kan ta litt tid, noen minutter å telle opp.

In [19]:
opptelling = tell_navn(korpus, redigerte_navn)
opptelling

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Lag en graf for en bok

Grafene kan legges sammen i etterkant om det ønskes en graf for hele korpuset.

In [ ]:
# fyll inn en indeks fra korpuslisten 0 er første URN, 1 neste osv.

G = tm.character_network(str(korpus['urn'][0]), redigerte_navn)
gnl.show_graph(G)

Dernest kan den vises og analyseres med passende kommandoer